# How have total sales and profit changed month-over-month?

In [0]:
use catalog superstore_databricks_dbt;

In [0]:

create schema if not exists superstore_analysis;
create or replace view superstore_analysis.monthly_growth as
with cte as (
  select
date_format(Order_Date, 'yyyy-MM') as year_month,
sum(Sales) as monthly_sales,
sum(profit) as profit,
lag(sum(Sales)) over (order by date_format(Order_Date, 'yyyy-MM')) as lag_monthly_sales,
lag(sum(profit)) over (order by date_format(Order_Date, 'yyyy-MM')) as lag_monthly_profit
from superstore_databricks_dbt.dbt_luo_gold.superstore_fact
group by year_month
order by year_month
)
select
year_month,
round(monthly_sales,2) as monthly_sales,
round(profit,2) as profit,
round((monthly_sales-lag_monthly_sales)/lag_monthly_sales,2) as sales_growth_rate,
round((profit-lag_monthly_profit)/lag_monthly_profit,2) as profit_growth_rate
from cte;
select * from superstore_analysis.monthly_growth


# Which region contributes the most to profit growth?

In [0]:
create or replace view superstore_analysis.region_contribution as
with monthly_total_profit as (
select
date_format(Order_Date, 'yyyy-MM') as year_month,
sum(profit) as monthly_profit
from superstore_databricks_dbt.dbt_luo_gold.superstore_fact
group by year_month
),
cte as (
select
date_format(fact.Order_Date, 'yyyy-MM') as year_month,
c.Region,
round(sum(fact.profit),2) as monthly_profit_per_region
from superstore_databricks_dbt.dbt_luo_gold.customer_dim as c
inner join superstore_databricks_dbt.dbt_luo_gold.superstore_fact as fact
on c.Customer_ID = fact.Customer_ID
group by year_month, Region
order by year_month, monthly_profit_per_region desc
), cte2 as(
  select
cte.year_month,
cte.Region,
cte.monthly_profit_per_region,
round(cte.monthly_profit_per_region/monthly_profit,2) as contribution_ratio,
row_number() over (partition by cte.year_month order by cte.monthly_profit_per_region desc) as rank
from cte
inner join monthly_total_profit
on cte.year_month = monthly_total_profit.year_month
order by year_month, contribution_ratio desc
)
select 
year_month,
Region,
monthly_profit_per_region,
contribution_ratio
from cte2
where rank = 1;

select * from superstore_analysis.region_contribution





# What are the top 10 customers by lifetime sales and profit?

In [0]:
create or replace view superstore_analysis.top_10_customers as
select
c.Customer_ID,
c.Customer_Name,
round(sum(f.Sales),2) as total_purchase
from superstore_databricks_dbt.dbt_luo_gold.customer_dim as c
left join superstore_databricks_dbt.dbt_luo_gold.superstore_fact as f
on c.Customer_ID = f.Customer_ID
group by c.Customer_ID, c.Customer_Name
order by total_purchase desc
limit 10;

select * from superstore_analysis.top_10_customers

# How does sales performance differ by segment (Consumer, Corporate, Home Office)?

In [0]:
create or replace view superstore_analysis.customer_segment as
with total_sales_profit as(
select
round(sum(sales),2) as total_sales,
round(sum(profit),2) as total_profit
from superstore_databricks_dbt.dbt_luo_gold.superstore_fact
), base as(
select
c.segment,
round(sum(f.Sales),2) as total_sales_per_segment,
round(sum(f.profit),2) as total_profit_per_segment
from superstore_databricks_dbt.dbt_luo_gold.customer_dim as c
left join superstore_databricks_dbt.dbt_luo_gold.superstore_fact as f
on c.Customer_ID = f.Customer_ID
group by c.segment
order by total_sales_per_segment desc, total_profit_per_segment desc
)
select
b.segment,
b.total_sales_per_segment,
b.total_profit_per_segment,
t.total_sales,
t.total_profit
from base as b
cross join total_sales_profit as t;

select * from superstore_analysis.customer_segment

Databricks visualization. Run in Databricks to view.